In [1]:
import sys
sys.path.append('../../src/')

from lib.pysoarlib import *
import os

# Current working directory (.../tutorials)
cwd = os.path.abspath('')

TODO: Summary. Something like this:

In this water jug problem, we will cover the following.
* Rules that initialize state, propose operators, and apply operators
* Logging state changes 
* Searching with heuristics

## 4.3 Initial State Creation

Here we'll write the first rule that *proposes* how to initialize state for the water jug
problem. Encoding state initialization as a proposal is recommended because often 
the agent has to choose between several tasks. Suggesting a proposal allows for the agent
to use control knowledge to pick among different operators that initialize state.


Convention is to seperate the rule name by asterisks.

`task*function(propose|apply|elaborate)*operator-name`


Now we'll write two rules: one rule to propose an operator, and one to apply an operator.

(Remember that an **operator** is something that performs an action in the external world or internally in the agent's mind)

In [2]:
# How we describe the proposal in English (we won't actually use this variable, just an example)
propose_initialize_water_jug_english = """
task = water-jug, propose, name = initialize-water-jug
If no task is selected,
then propose the initialize-water-jug operator.
"""

propose_initialize_water_jug = """
sp {water-jug*propose*initialize-water-jug
    (state <s> ^superstate nil)
   -(<s> ^name)
-->
    (<s> ^operator <o> +)
    (<o> ^name initialize-water-jug)}
"""

The order of the conditions doesn't matter **except** that the first condition 
must be some sort of state test (like, is this the idle state, is this the <some task> state, etc).

`-(<s> ^name)` tests that the attribute `^name` is not on the state `<s>`.

In [3]:
# This is another valid to write the rule above
propose_initialize_water_jug_compact = """
sp {water-jug*propose*initialize-water-jug
    (state <s> ^superstate nil
              -^name)
-->
    (<s> ^operator <o> +)
    (<o> ^name initialize-water-jug)}
"""

Now we can write the rule that actually performs actions to working memory to initialize the state. So far we've only proposed to Soar that we *could* initialize the water jug task state.

We'll set the jug contents to 0 and add the attribute `^name` to state.

In [4]:
initialize_water_jug = """

sp {water-jug*apply*initialize-water-jug
    (state <s> ^operator <o>)
    (<o> ^name initialize-water-jug)
-->
    (<s> ^name water-jug ^jug <j1>
         ^jug <j2>) 
    (<j1> ^volume 5
          ^contents 0) 
    (<j2> ^volume 3
          ^contents 0)}

"""

Notice that we're not adding the `^empty` attribute now. 
This will be set by another rule that computes the value from `^volume` and `^contents`.


Again, there's a more compact way to write the condition for matching on the operator name. 
Above, `<o>` is used to connect the first condition to the second. In Soar, we can combine conditions that are linked by variables. 
We just replace intermediate variables with a period. 

In [5]:
initialize_water_jug_compact = """

sp {water-jug*apply*initialize-water-jug
    (state <s> ^operator.name initialize-water-jug)
-->
    (<s> ^name water-jug ^jug <j1>
         ^jug <j2>) 
    (<j1> ^volume 5
          ^contents 0) 
    (<j2> ^volume 3
          ^contents 0)}

"""

## 4.4 Persistence of Working Memory Elements

After the `water-jug*apply*initialize-water-jug` rule fires, the task name `water-jug` and the two jugs are added to working memory. 
Now notice that the condition in our propsal rule no longer matches.

```
sp {water-jug*propose*initialize-water-jug
    (state <s> ^superstate nil
              -^name)
    ...
```

This is because this rule looks for the absence of the `^name` attribute on some state `<s>`.
Because we have just added `^name` to state, the proposal is now removed from working memory.


Now that the proposal is removed from memory (aka *retracted*), what happens to the WM elements that we just added?
It would be a bit silly for Soar to add them to WM then immediately remove them.

Soar resolves this issue by allowing operator rules to modify state in a **persistent** manner and have **operator-support**. 
All other rules don't actually modify the current state.  They compute *entailments*/*elaborations* of the current situation and have **instantiation support**. 

TODO: this is confusing
> Soar makes a distinction between the persistence of working memory elements created by operator application rules and the persistence of working memory elements created by other types of rules. Operator application needs to create persistent results because operators are the deliberate commitments that the system has made – it has decided to apply the operator and make potentially non-monotonic changes to the state – to move itself to a new state in the problem space. All other rules in Soar compute the entailments/elaborations of the current state without actually changing it and should be retracted when they no longer match the current state.

(TODO cont.) It seems like the below `elaboration` and `application` rules both modify state in the same way. Especially confused by "All other rules in Soar compute the entailments/elaborations of the current state without actually changing it".

`operator support (o-support)` - WM elements that are created as part of an operator and will persist until they are removed by another operator or become disconnected from the state.

`instantiation support (i-support)` - WM elements will persist only as long as the rule instatntiation that created them still matches.

A concrete example of the `o-support` is that when we *apply* the rule to pour water from jugA to jugB, we want the WM to reflect these changes.

For `i-support`, proposal rules inherently fall into this camp. 
We want to propose rules only when we directly match a condition.
If that condition changes, we want that rule to not be applied anymore.

We'll see in the next section a case for `i-support` where we want to keep track of how `empty` a jug is by watching for `contents` and `volume` state changes and modifying the jug to reflect it's amount empty is volume - contents.

TODO Move to a better place maybe?: "A rule is an operator application rule if it tests a selected operator and modifies the state.
Persistent working memory elements are said to be *operator-supported*, because they are created by operators."

TODO good quote, probably should find a better place for this

> Persistence is necessary for creating memories of prior events, such as the memory of something sensed in the environment. For example, if an eater needed to remember the location of uneaten food so that it could return to it later, the eater would have to create a persistent structure in its working memory.

## 4.5 Water Jug State Elaboration

If you recall, we haven't actually added the `^empty` attribute to working memory yet. 

`^empty` is a result of `^volume` - `^contents`. If we were to add this rule using the mechanisms we've learned about so far, we would have to maintain this subtraction in every operator application rule.

That method is very cumbersome and would lead to a lot of extra code. 
What we want is a rule that independently checks for a state with a `^jug` and adds the
`^empty` attribute.

In Soar, we can use a **state elaboration rule**.

State elaboration rules test states and potentially create new structures on the state.

These rules are *i-supported* meaning that when parts of WM chage, the structures added by the state elaboration rules are automatically updated.


> **Note** If you come from a reactive programming background (like Svelte), it may help to think
of this as *subscribing* to jugs, and mapping over them, adding the `empty` property.
TODO Revisit << also feels a bit like ECS in a way


#### Writing the rule

An english representation of the state elaboration rule to compute `empty`.


```
task = water-jug, elaborate, operator name = empty
If the state is named 'water-jug' and a jug can hold volume 'v'
and currently has the contents c,
then add that it has (v - c) available 'empty' space
```

Soar allows you do do math in rules. The authors seem to have taken some inspiration from functional programming syntax seen in languages like lisp.

To write (volume - contents) in soar, you write the operation first, followed by the arguments: (- volume contents). 
We can nest math operations by using additional parenthesis (+ volume (* 2 contents)).

Now let's write the rule that will automatically update the jug state when the value of `^volume` or `^contents` changes through an operator application rule (TODO this can't be simulated with i-support rules?).

In [6]:
elaborate_empty_jug = """

sp {water-jug*elaborate*empty 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^volume <v>
         ^contents <c>) 
-->
    (<j> ^empty (- <v> <c>))}

"""

## 4.6 Water Jug State Initialization and Elaboration



### Running it

Let's see the result of our progress so far. We'll use the `watch` command to trace

The `watch` command can have one of the following levels
```
0 watch nothing
1 watch decisions
2 watch phases, gds, and decisions
3 watch productions, phases, and decisions
4 watch wmes, productions, phases, and decisions
5 watch preferences, wmes, productions, phases, and decisions
```

In [7]:
agent_raw = f"""
{propose_initialize_water_jug}
{initialize_water_jug}
{elaborate_empty_jug}
"""

jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

# can also add this to the SoarAgent constructor: watch_level=5
# reference: soar manual p.167
jug_agent.execute_command("watch --level 5");

--------- SOURCING PRODUCTIONS ------------
Total: 3 productions sourced.


We can step through the execution using the `step` command.

The first thing that happens is that the `initialize-water-jug` propoal's condition matches.
We fire that rule.

In [8]:
jug_agent.execute_command("step", print_res=True);

--- Input Phase ---
--- END Input Phase ---
--- Proposal Phase ---

--- Inner Elaboration Phase, active level 1 goal S1 ---

Firing water-jug*propose*initialize-water-jug
+ (O1 ^name initialize-water-jug +) (water-jug*propose*initialize-water-jug)
+ (S1 ^operator O1 +) (water-jug*propose*initialize-water-jug)
=>WM: (15: S1 ^operator O1 +)
=>WM: (14: O1 ^name initialize-water-jug)

--- END Proposal Phase ---
--- Decision Phase ---
=>WM: (16: S1 ^operator O1)

     1:    O: O1 (initialize-water-jug)
--- END Decision Phase ---
step
--> 1 decision cycle executed. 1 rule fired.


#### Soar Cycle

Remember that Soar's cycle consists of three parts.

![soar cycle](img/soar-cycle-3-steps.png)

1. Propose operators. Here rules fire to propose operators. This is what we just witnessed.

    Proposal rules test features of the state to ensure that the operator is appropriate.
    If it is, it is added to WM with an *acceptable preference* (TODO explain).

    *In context:*  We see two things have been added to working memory.
    ```
    =>WM: (S1 ^operator O1 +)
    =>WM: (O1 ^name initialize-water-jug)
    ```

    The `+` signifies that the operator is proposed and not selected.

2. Next, the decision procedure selects the current operator. (TODO explain how briefly)

    *In context:*  In the decision phase we see that we have added the attribute `^operator` with the value `initialize-water-jug` to state `S1`. 
    We have selected the water jug initialization operator. 

3. Finally we apply the operator.

    This is what we see on the next `step`.

In [9]:
jug_agent.execute_command("step", print_res=True);

--- Application Phase ---
	--- Firing Productions (PE) For State At Depth 1 ---

--- Inner Elaboration Phase, active level 1 goal S1 ---

Firing water-jug*apply*initialize-water-jug
+ (J2 ^contents 0 +  :O ) (water-jug*apply*initialize-water-jug)
+ (J2 ^volume 3 +  :O ) (water-jug*apply*initialize-water-jug)
+ (J1 ^contents 0 +  :O ) (water-jug*apply*initialize-water-jug)
+ (J1 ^volume 5 +  :O ) (water-jug*apply*initialize-water-jug)
+ (S1 ^jug J2 +  :O ) (water-jug*apply*initialize-water-jug)
+ (S1 ^jug J1 +  :O ) (water-jug*apply*initialize-water-jug)
+ (S1 ^name water-jug +  :O ) (water-jug*apply*initialize-water-jug)
	--- Change Working Memory (PE) ---
=>WM: (23: J2 ^contents 0)
=>WM: (22: J2 ^volume 3)
=>WM: (21: J1 ^contents 0)
=>WM: (20: J1 ^volume 5)
=>WM: (19: S1 ^jug J1)
=>WM: (18: S1 ^jug J2)
=>WM: (17: S1 ^name water-jug)
	--- Firing Productions (IE) For State At Depth 1 ---

--- Inner Elaboration Phase, active level 1 goal S1 ---

Firing water-jug*elaborate*empty

Firing w

A lot seemingly happens in this application phase, but at the end of the day, Soar has just added a bunch of persistent state to WM. We add the jugs, their volume and contents, and the task name.


After, our `elaborate*empty` rule fires to add the `^empty` attribute to state.
```
Firing water-jug*elaborate*empty
Firing water-jug*elaborate*empty
+ (J2 ^empty 3 +) (water-jug*elaborate*empty)
+ (J1 ^empty 5 +) (water-jug*elaborate*empty)
```
This rule fires twice because we have two jugs. 
Each match to the rule is called an `instantiation` (a set of WM elements that successfully match a rule). The rules fire in parallel.


Because the operator proposal was *i-support* memory, it is removed after it's conditions no longer match the current state. 
```
Retracting water-jug*propose*initialize-water-jug
- (O1 ^name initialize-water-jug +) (water-jug*propose*initialize-water-jug)
- (S1 ^operator O1 +) (water-jug*propose*initialize-water-jug)
	--- Change Working Memory (IE) ---

        REMOVING: Operator from context slot (proposal no longer matches): (15: S1 ^operator O1 +)
=>WM: (25: J2 ^empty 3)
=>WM: (24: J1 ^empty 5)
<=WM: (15: S1 ^operator O1 +)
<=WM: (16: S1 ^operator O1)
<=WM: (14: O1 ^name initialize-water-jug)
```

## Soar's cycle elaborated

Let's take another look at the cycle.

![soar cycle](img/soar-cycle-5-steps.png)

We don't have any IO in this problem, so we can ignore the first and last box.

In the `Elaborate State / Propose Operators` box, Soar elaborates state or proposes operators.

Soar wants to ensure that it uses all of it's available knowledge. To ensure this, Soar runs proposal and elaboration rules repeatedly (in parallel) until there are no rules left to run. These rules will either fire (doing things like creating new state) or retract (becoming irrelevant because their conditions don't match the current state).

Soar enters the `Select Operator` phase when there are no rules left to fire. This phase was referred to earlier as the *decision procedure*.

Once Soar selects an operator, it then fires the rules that apply the operator. Often, more rules will be proposed and more state will be elaborated in this phase.

Once there are no more rules left to fire, Soar moves on to the output phase. The process is then repeated, starting at the input phase.

TODO is it a bad practice to modify state (other than the operator) in a propose operator?

## 4.7 Water Jug Operator Proposals

We have to write three operators: fill, empty, an pour.

TODO Rewrite following two:

can apply: proposal rules that create acceptable preferences for operators

should apply: search control rules that create other types of preferences

Conditions for our operators
- **Fill** If a jug is not full, fill it with water from the well.
- **Empty** If there is water in the jug, empty it into the well
- **Pour** If there is water in the source jug and the destination jug is not full, pur water from souce to destination


Soar operators *must* change the state in some way in order to be selected. Therefore we can't allow a full jug to be filled, as that wouldn't change the state.

TODO Why *must* soar change the state?


Psuedo SML
```
task = water-jug, propose, operator name = fill
If the task is water-jug and there is a jug that is not full, then propose filling that jug.

task = water-jug, propose, operator name = empty
If the task is water-jug and there is a jug that is not empty, then propose emptying that jug.

task = water-jug, propose, operator name = pour
If the task is water-jug and there is a jug that is not full and the other jug is not empty,
then propose pouring water from the second jug into the first jug.
```


Keeping in mind the restriction mentioned above, we can write the fill proposal.

In [10]:
propose_fill = """

sp {water-jug*propose*fill 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> -^empty 0)
-->
    (<s> ^operator <o> +) 
    (<o> ^name fill
         ^fill-jug <j>)}

"""

`(<j> -^empty 0)` ensures that the jug we want to fill does not have exactly 0 space left. (Remember `^empty` is not a boolean, but keeps track of how many gallons can fill a jug).


We could also write that condition using a comparison operator.

`(<j> ^empty > 0)`. This is arguably easier to understand.

We can't simply do `volume - contents > 0` because soar does not allow math within its conditions. That is why we created a elaboration to handle the math.

#### Exercise

Try writing the condition for the empty operator. 

```
sp {water-jug*propose*empty 
    (state <s> ^name water-jug
               ^jug <j>) 
    **** Another condition goes here ****
-->
    (<s> ^operator <o> +) 
    (<o> ^name empty
         ^empty-jug <j>)}
```

Similarily, write the conditions to complete the pour proposal.

```
sp {water-jug*propose*pour 
    *** Variables jug1 and jug2 are actually jugs ***
    *** Contents condition ***
    *** Empty condition ***
-->
    (<s> ^operator <o> +) 
    (<o> ^name pour
         ^empty-jug <i> 
         ^fill-jug <j>)}
```


The answers are in the following cell.

---

>

>

>


### Intentional Blank Space to Hide Answers

>

>

>

---

In [11]:
propose_empty = """

sp {water-jug*propose*empty 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^contents > 0)
-->
    (<s> ^operator <o> +) 
    (<o> ^name empty
         ^empty-jug <j>)}
         
"""

propose_pour = """

sp {water-jug*propose*pour 
    (state <s> ^name water-jug
            ^jug <i>
            ^jug { <j> <> <i> })
    (<i> ^contents > 0)
    (<j> ^empty > 0)
-->
    (<s> ^operator <o> +) 
    (<o> ^name pour
         ^empty-jug <i> 
         ^fill-jug <j>)}

"""

So what was up with this condition?

```
(state <s> ^name water-jug
           ^jug <i>
           ^jug { <j> <> <i> })
```

Soar allows variables in WM to match themselves. So the conditions `(<i> ^contents > 0)` and `(<j> ^empty > 0)` can match the same jug, unless we add a restriction that `<i>` **cannot** be `<j>`.

If you wrote something like this...
```
(state <s> ^name water-jug
           ^jug <i>
           ^jug <j>)
```
...you were close.

In order to ensure `<i>` is a jug, `<j>` is a jug, AND `<i>` != `<j>`, we must use the `<>` **predicate**.

### Predicates

When testing an identifier, attribute, or value, we can use predicates to do more complex tests. These predicated **must** come before the identifier/attribute/value they are modifying.

![predicates table](img/predicates-simple-table.png)

Take this following example.

```
sp {propose-operator*to-show-example-predicate 
    (state <s> ^car <c>)
    (<c> ^style convertible ^color <> rust)
-->
    (<s> ^operator <o> +)
    (<o> ^name drive-car ^car <c>) }
```

"In this production, there must be a “color” attribute for the working memory object that matches <c>, and the value of that attribute must not be “rust”." Soar Manual p.43.


`(<car> ^color <> rust)` ... attribute color != 'rust'.


#### Disjunctions
    
What if we want to write a condition where the car's color can be 'yellow' *or* 'green'? We use **disjunctions**. We test disjunctions with double angle brackets ``<< or >>``. TODO manual has `<< and >>` is that a typo?


`(<car> ^color << yellow green >>)` ... attribute color == 'yellow' OR color == 'green'.


#### Conjunctions    
    
Finally, we can also test **conjunctions**, meaning all of the conditions for a identifier/attribute/value must hold. We use curly braces to denote conjunctions (`{ and }`).


If we wanted to filter for cars with yellow *and* green paint, we could do the following
`(<car> ^color { yellow green })`. 

We can also use equality operators in these statements (`>`, `<`, `<=`, `>=`).

A car with a top speed greater than `50` *and* less than some variable speed limit.

`(<car> ^current-speed { > 50 < <speed-limit> })`


### Back to the water jug

Now we can finally understand this condition.

```
(state <s> ^name water-jug
           ^jug <i>
           ^jug { <j> <> <i> })
```

We're filtering for a value `<i>` that is a jug on state `<s>`. And another jug that meets the two conditions: is a jug (`<j>`) *and* is not equal to `<i>` (`<> <i>`).


#### Exercise

Label each of the following as `valid` syntax or `invalid` syntax.

1. `> <valuex>`
2. `> > <valuey>` 
3. `<=> <y>` 
4. `1 >`
5. `< 1` 
6. `= 10`
7. `<< good-morning good-evening >>`
8. `<< < 5 > 10 >>`
9. `<< 5 10 >>`
10. `<< <A> A >>`
11. `<< A B C 45 I17 >>`
12. `<<A B C >>`

Answers can be found in the Soar manual on pages 43-45.

## Running the agent

Let's step through what we have so far.

In [12]:
agent_raw = f"""
{propose_initialize_water_jug}
{initialize_water_jug}
{elaborate_empty_jug}
# New rules
{propose_empty}
{propose_pour}
{propose_fill}
"""

jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

jug_agent.execute_command("watch --level 3")
jug_agent.execute_command("step");

--------- SOURCING PRODUCTIONS ------------
Total: 6 productions sourced.
--- Input Phase ---
--- END Input Phase ---
--- Proposal Phase ---

--- Inner Elaboration Phase, active level 1 goal S1 ---

Firing water-jug*propose*initialize-water-jug

--- END Proposal Phase ---
--- Decision Phase ---

     1:    O: O1 (initialize-water-jug)
--- END Decision Phase ---


No new things to talk about here. The Soar agent just proposes to initialize the water jug.

In [13]:
jug_agent.execute_command("step");

--- Application Phase ---
	--- Firing Productions (PE) For State At Depth 1 ---

--- Inner Elaboration Phase, active level 1 goal S1 ---

Firing water-jug*apply*initialize-water-jug
	--- Change Working Memory (PE) ---
	--- Firing Productions (IE) For State At Depth 1 ---

--- Inner Elaboration Phase, active level 1 goal S1 ---

Firing water-jug*elaborate*empty

Firing water-jug*propose*fill

Firing water-jug*elaborate*empty

Firing water-jug*propose*fill

Retracting water-jug*propose*initialize-water-jug
	--- Change Working Memory (IE) ---

        REMOVING: Operator from context slot (proposal no longer matches): (15: S1 ^operator O1 +)
	--- Firing Productions (IE) For State At Depth 1 ---

--- Inner Elaboration Phase, active level 1 goal S1 ---
	--- Change Working Memory (IE) ---

--- END Application Phase ---

--- Output Phase ---
--- END Output Phase ---
--- Input Phase ---
--- END Input Phase ---
--- Proposal Phase ---

--- END Proposal Phase ---
--- Decision Phase ---

     2:   

In the Inner Elaboration Phase you can see that the state elaboration that maintains the `^empty` attribute is fired twice, once for each jug. 
Following each fire is a proposal for the `fill` rule.

Then if you scroll to the very bottom you'll see this, `==>S: S2 (operator tie)`. Two fills were proposed, but which one does Soar choose? Well, in this situation it can't, so a *tie **impasse*** is created. I like to think of *impasses* as exceptions. Soar throws an exception when it doesn't know what to do next. To solve *impasses* Soar creates a **substate** to try to solve the issue. *Substates* are discussed in later chapters.

TODO if this just keeps running, it just gets stuck in an infinite loop. ~~Is this because substates are being created over and over again?~~ It also does this with the indifferent operator added. Why?

If we step through Soar again, nothing happens. If you run Soar, it will go into an infinite loop (and eventually stop itself, raising an error to the user).

In [14]:
jug_agent.execute_command("step");

--- Application Phase ---

--- END Application Phase ---

--- Output Phase ---
--- END Output Phase ---
--- Input Phase ---
--- END Input Phase ---
--- Proposal Phase ---

--- END Proposal Phase ---
--- Decision Phase ---

     3:       ==>S: S3 (state no-change)
--- END Decision Phase ---


To solve this issue, we're going to assign another *preference* to the operator. We already had the `+` preference which indicated it was *acceptable*, meaning it is a candidate for selection. This is the most basic preference that must be assigned to anything that you want to be selected*.

However, we have two proposed rules with the same *acceptable* preference. We can add another preference to the `fill` operator, the *indifferent* preference (`=`). When Soar sees this, it just chooses randomly among all the alternatives.

See manual p.19 for a full list of preferences.

\* Except for items with a *require* preference.

In [15]:
propose_fill_indifferent_verbose = """

sp {water-jug*propose*fill 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^empty > 0)
-->
    (<s> ^operator <o> +
         ^operator <o> =) 
    (<o> ^name fill
         ^fill-jug <j>)}

"""

# We can shorten the syntax a bit
propose_fill_indifferent = """

sp {water-jug*propose*fill 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^empty > 0)
-->
    (<s> ^operator <o> + =)
    (<o> ^name fill
         ^fill-jug <j>)}

"""

# We'll make all of the operators indifferent 
propose_empty_indifferent = """

sp {water-jug*propose*empty 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^contents > 0)
-->
    (<s> ^operator <o> + =) 
    (<o> ^name empty
         ^empty-jug <j>)}
         
"""

propose_pour_indifferent = """

sp {water-jug*propose*pour 
    (state <s> ^name water-jug
            ^jug <i>
            ^jug { <j> <> <i> })
    (<i> ^contents > 0)
    (<j> ^empty > 0)
-->
    (<s> ^operator <o> + =) 
    (<o> ^name pour
         ^empty-jug <i> 
         ^fill-jug <j>)}

"""

In [16]:
agent_raw = f"""
{propose_initialize_water_jug}
{initialize_water_jug}
{elaborate_empty_jug}
# Changed rules
{propose_empty_indifferent}
{propose_pour_indifferent}
{propose_fill_indifferent}
"""

jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

jug_agent.execute_command("watch --level 1")
jug_agent.execute_command("step 4"); # step 4 times

--------- SOURCING PRODUCTIONS ------------
Total: 6 productions sourced.
1:    O: O1 (initialize-water-jug)
2:    O: O2 (fill)
3:    ==>S: S2 (operator no-change)
4:       ==>S: S3 (state no-change)


## 4.8 Operator Application

We want to simulate the pouring of water. To this, we will modify the `^contents` of each jug. As a result, `^empty` will also update.

Let's write the *apply* `fill` rule in english.
```
task = water-jug, apply, operator name = fill
If the task is water-jug and the fill operator is selected for a given jug, 
then set that jug’s contents to be its volume.
```

In Soar, we can't simply modify a value in WM. So, instead of updating the value of contents, we have to delete the old WM element and create a new one.

Deleting a WM element is the same as adding one, but we must add `-` to the end of the statement.

In [17]:
apply_fill = """

sp {water-jug*apply*fill 
    (state <s> ^name water-jug
               ^operator <o> ^jug <j>)
    (<o> ^name fill ^fill-jug <j>)
    (<j> ^volume <volume> ^contents <contents>)
-->
    (<j> ^contents <volume>)
    (<j> ^contents <contents> -)}

"""

Now for the empty operator application

```
If the task is water-jug and the empty operator is selected for a given jug, 
then set that jug’s contents to be 0 and its empty to be its volume.
```

In [18]:
apply_empty = """

sp {water-jug*apply*empty 
    (state <s> ^name water-jug
               ^operator <o>
               ^jug <j>) 
    (<o> ^name empty
         ^empty-jug <j>) 
    (<j> ^volume <volume>
         ^contents <contents>) 
-->
    # Compact way to create and delete in one identifier
    (<j> ^contents 0
         ^contents <contents> -)}

"""

Pouring is the most complicated.

When pouring from a full 5-gallon jug to a 3-gallon jug, the 5-gallon jug still has 2-gallons left. 

When pouring from a full 3-gallon jug to a 5-gallon jug, the 3-gallon jug is completely emptied, and the 5-gallon jug only has 3-gallons in it.

We have to write two rules for these situations.

**If we will empty the jug**
```
If the task is water-jug and the pour operator is selected,
    and the contents of the jug being emptied are less than or equal to
    the empty amount of the jug being filled,
then set the contents of the jug being emptied to 0;
    set the contents of the jug being filled to the sum of the two jugs.
```

**If we will won't empty the jug**
```
If the task is water-jug and the pour operator is selected, 
    and the contents of the jug being emptied are greater than the empty amount
    of the jug being filled,
then set the contents of the jug being emptied to its contents minus the
    empty of the jug being filled;
    set the contents of the jug filled to its volume.
```

In [19]:
apply_pour__will_empty_jug = """

sp {water-jug*apply*pour*will-empty-empty-jug 
    (state <s> ^name water-jug
               ^operator <o>) 
    (<o> ^name pour
         ^empty-jug <source>
         ^fill-jug <dest>) 
    (<dest> ^volume <dest-volume>
            ^contents <dest-contents>
            ^empty <dest-empty>) 
    # Main condition to test
    (<source> ^volume <source-vol>
              ^contents { <source-contents> <= <dest-empty> }) 
-->
    (<source> ^contents 0 ^contents <source-contents> -)
    (<dest> ^contents (+ <dest-contents> <source-contents>) ^contents <dest-contents> -)}

"""

apply_pour__wont_empty_jug = """

sp {water-jug*apply*pour*will-not-empty-empty-jug 
    (state <s> ^name water-jug
               ^operator <o>) 
    (<o> ^name pour
         ^empty-jug <source>
         ^fill-jug <dest>)
    # Main condition to test
    (<source> ^volume <source-vol>
              ^contents { <source-contents> > <dest-empty> })
    (<dest> ^volume <dest-vol>
            ^contents <dest-con>
            ^empty <dest-empty>)
-->
    (<source> ^contents (- <source-contents> <dest-empty>) 
              ^contents <source-contents> -)
    # Update destination jug's contents to be it's capacity
    (<dest> ^contents <dest-vol> 
            ^contents <dest-con> -)}

"""

We're almost ready to run the program, but we should discuss monitoring first so that you can have an easier time following the execution path.

## 4.9 State and Operator Monitoring

We can add more rules to the runtime of Soar to monitor state and operators. We'll use the same syntax we've learned above to match on state that we're interested in and print it out.

In this first rule, we'll match on two jugs with the volumes of 3 and 5 and print them out.

Note `crlf` is making a new line (like `\n` in most languages).

Text between the pipes `| text |` is printed as is (it's not interpreted by Soar as a command).

In [20]:
monitor_jugs = """

sp {water-jug*monitor*jugs 
    (state <s> ^name water-jug
               ^jug <i> <j>)
    (<i> ^volume 3 ^contents <icon>)
    (<j> ^volume 5 ^contents <jcon>)
-->
    (write (crlf) | 3:| <icon> | 5:| <jcon>)}

"""

We can write extra rules for monitoring the application of the fill, empty, and pour operators.

(Recall the `empty-jug` is an attribute we added to the `empty` operator that keeps track of the jug we want to empty)

TODO unsure why I can't add two new lines to the above statement (`(write (crlf) | 3:| <icon> | 5:| <jcon> (crlf) (crlf) )}`)

In [21]:
monitor_empty_application = """

sp {water-jug*monitor*operator-application*empty 
    (state <s> ^name water-jug
               ^operator <o>) 
    (<o> ^name empty
         ^empty-jug.volume <volume>) 
-->
    (write | EMPTY(| <volume> |)|)}

"""

monitor_fill_application = """

sp {water-jug*monitor*operator-application*fill 
    (state <s> ^name water-jug
               ^operator <o>) 
    (<o> ^name fill
         ^fill-jug.volume <volume>) 
-->
    (write | FILL(| <volume> |)|)}

"""

monitor_pour_application = """

sp {water-jug*monitor*operator-application*pour 
    (state <s> ^name water-jug
               ^operator <o>) 
    (<o> ^name pour
        ^empty-jug <i>
        ^fill-jug <j>)
    (<i> ^volume <ivol> ^contents <icon>)
    (<j> ^volume <jvol> ^contents <jcon>)
-->
    (write | POUR(| <ivol> |:| <icon> |,| <jvol> |:| <jcon> |)|)}

"""

In [22]:
agent_raw = f"""
{propose_initialize_water_jug}
{initialize_water_jug}
{elaborate_empty_jug}
{propose_empty_indifferent}
{propose_pour_indifferent}
{propose_fill_indifferent}
# Added rules
{apply_empty}
{apply_fill}
{apply_pour__will_empty_jug}
{apply_pour__wont_empty_jug}
{monitor_jugs}
{monitor_empty_application}
{monitor_fill_application}
{monitor_pour_application}
"""

jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

jug_agent.execute_command("watch --level 1")
jug_agent.execute_command("step 20"); # step 4 times

--------- SOURCING PRODUCTIONS ------------
Total: 14 productions sourced.
1:    O: O1 (initialize-water-jug)
 3:0 5:0
2:    O: O3 (fill) FILL(3)
 3:3 5:0
3:    O: O2 (fill) FILL(5)
 3:3 5:5
4:    O: O6 (empty) EMPTY(5)
 3:3 5:0
5:    O: O5 (empty) EMPTY(3)
 3:0 5:0
6:    O: O9 (fill) FILL(3)
 3:3 5:0
7:    O: O8 (fill) FILL(5)
 3:3 5:5
8:    O: O11 (empty) EMPTY(3)
 3:0 5:5
9:    O: O14 (fill) FILL(3)
 3:3 5:5
10:    O: O12 (empty) EMPTY(5)
 3:3 5:0
11:    O: O15 (empty) EMPTY(3)
 3:0 5:0
12:    O: O18 (fill) FILL(3)
 3:3 5:0
13:    O: O17 (fill) FILL(5)
 3:3 5:5
14:    O: O21 (empty) EMPTY(5)
 3:3 5:0
15:    O: O20 (empty) EMPTY(3)
 3:0 5:0
16:    O: O24 (fill) FILL(3)
 3:3 5:0
17:    O: O25 (pour) POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
18:    O: O27 (empty) EMPTY(5)
 3:0 5:0
19:    O: O30 (fill) FILL(3)
 3:3 5:0
20:    O: O31 (fill)


As you may have noticed, this agent isn't really guided in their attentions. It's a bit random. And if you watch for long enough, you may see the agent get to the goal... but it'll keep going. We need a way to tell the agent we *desire* for it to reach some state.

## 4.10 Desired State Recognition

This desired state rule will see if the 3-gallon jug has 1 gallon in it, print the result, and halt. Let's write it.

In [23]:
detect_goal_state = """

sp {water-jug*detect*goal*achieved 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^volume 3
         ^contents 1)
-->
    (write (crlf) |The problem has been solved.|) 
    (halt)}

"""

TODO: maybe dumb question but is there a reason we'd do a proposal first?

Notice that we hardcoded "jug of volume 3 with the contents 1" as a condition. Another way to deal with goal states is by creating a WM element and matching that instead. For example, this would be beneficial in the scenario where we want either a "jug of volume X with contents Y" or "jug of volume A with contents B".

(This is also discussed later during a section on *means end analysis*.)

Let's create a rule that'll create the goal state during initialization. This rule will execute in parallel along side the other initialization rule.

In [24]:
initialize_goal_state = """

sp {water-jug*apply*initialize*create*desired-state 
    (state <s> ^operator.name initialize-water-jug) 
-->
    (<s> ^desired.jug <k>)
    (<k> ^volume 3 ^contents 1)}

"""

detect_goal_state = """

sp {water-jug*detect*goal*achieved 
    (state <s> ^name water-jug
               ^desired.jug <desired>
               ^jug <j>) 
    (<j>       ^volume <v> ^contents <c>)
    (<desired> ^volume <v> ^contents <c>)
-->
    (write (crlf) |The problem has been solved.|) 
    (halt)}

"""

In [25]:
agent_raw = f"""
# Initialization
{initialize_water_jug}
# Added rule
{initialize_goal_state}
# Elaborations
{elaborate_empty_jug}
# Proposals
{propose_initialize_water_jug}
{propose_empty_indifferent}
{propose_pour_indifferent}
{propose_fill_indifferent}
# Application
{apply_empty}
{apply_fill}
{apply_pour__will_empty_jug}
{apply_pour__wont_empty_jug}
# Monitor
{monitor_jugs}
{monitor_empty_application}
{monitor_fill_application}
{monitor_pour_application}
# Added rule
{detect_goal_state}
"""

jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

jug_agent.execute_command("run");

--------- SOURCING PRODUCTIONS ------------
Total: 16 productions sourced.
1:    O: O1 (initialize-water-jug)
3:0 5:0
2:    O: O3 (fill)
FILL(3)
 3:3 5:0
3:    O: O5 (empty)
EMPTY(3)
 3:0 5:0
4:    O: O6 (fill)
FILL(3)
 3:3 5:0
5:    O: O2 (fill)
FILL(5)
 3:3 5:5
6:    O: O8 (empty)
EMPTY(3)
 3:0 5:5
7:    O: O11 (fill)
FILL(3)
 3:3 5:5
8:    O: O12 (empty)
EMPTY(3)
 3:0 5:5
9:    O: O13 (pour)
POUR(5:5,3:0) POUR(5:2,3:3)
 3:3 5:2
10:    O: O15 (empty)
EMPTY(3)
 3:0 5:2
11:    O: O21 (fill)
FILL(3)
 3:3 5:2
12:    O: O23 (empty)
EMPTY(3)
 3:0 5:2
13:    O: O19 (fill)
FILL(5)
 3:0 5:5
14:    O: O26 (pour)
POUR(5:5,3:0) POUR(5:2,3:3)
 3:3 5:2
15:    O: O32 (fill)
FILL(5)
 3:3 5:5
16:    O: O33 (empty)
EMPTY(5)
 3:3 5:0
17:    O: O35 (fill)
FILL(5)
 3:3 5:5
18:    O: O28 (empty)
EMPTY(3)
 3:0 5:5
19:    O: O38 (fill)
FILL(3)
 3:3 5:5
20:    O: O39 (empty)
EMPTY(3)
 3:0 5:5
21:    O: O40 (pour)
POUR(5:5,3:0) POUR(5:2,3:3)
 3:3 5:2
22:    O: O44 (empty)
EMPTY(5)
 3:3 5:0
23:    O: O42 (empt

 3:0 5:5
190:    O: O395 (empty)
EMPTY(5)
 3:0 5:0
191:    O: O393 (fill)
FILL(3)
 3:3 5:0
192:    O: O398 (empty)
EMPTY(3)
 3:0 5:0
193:    O: O399 (fill)
FILL(3)
 3:3 5:0
194:    O: O401 (empty)
EMPTY(3)
 3:0 5:0
195:    O: O396 (fill)
FILL(5)
 3:0 5:5
196:    O: O402 (fill)
FILL(3)
 3:3 5:5
197:    O: O404 (empty)
EMPTY(5)
 3:3 5:0
198:    O: O406 (pour)
POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
199:    O: O411 (fill)
FILL(3)
 3:3 5:3
200:    O: O408 (empty)
EMPTY(5)
 3:3 5:0
201:    O: O415 (fill)
FILL(5)
 3:3 5:5
202:    O: O413 (empty)
EMPTY(3)
 3:0 5:5
203:    O: O417 (pour)
POUR(5:5,3:0) POUR(5:2,3:3)
 3:3 5:2
204:    O: O422 (pour)
POUR(3:3,5:2) POUR(3:0,5:5)
 3:0 5:5
205:    O: O424 (empty)
EMPTY(5)
 3:0 5:0
206:    O: O427 (fill)
FILL(5)
 3:0 5:5
207:    O: O426 (fill)
FILL(3)
 3:3 5:5
208:    O: O429 (empty)
EMPTY(5)
 3:3 5:0
209:    O: O432 (fill)
FILL(5)
 3:3 5:5
210:    O: O433 (empty)
EMPTY(5)
 3:3 5:0
211:    O: O435 (fill)
FILL(5)
 3:3 5:5
212:    O: O430 (empty)
EMPTY(3)


536:    O: O1104 (fill)
FILL(5)
 3:3 5:5
537:    O: O1107 (empty)
EMPTY(5)
 3:3 5:0
538:    O: O1106 (empty)
EMPTY(3)
 3:0 5:0
539:    O: O1109 (fill)
FILL(5)
 3:0 5:5
540:    O: O1112 (empty)
EMPTY(5)
 3:0 5:0
541:    O: O1110 (fill)
FILL(3)
 3:3 5:0
542:    O: O1114 (pour)
POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
543:    O: O1119 (fill)
FILL(3)
 3:3 5:3
544:    O: O1121 (empty)
EMPTY(3)
 3:0 5:3
545:    O: O1117 (fill)
FILL(5)
 3:0 5:5
546:    O: O1123 (fill)
FILL(3)
 3:3 5:5
547:    O: O1125 (empty)
EMPTY(5)
 3:3 5:0
548:    O: O1128 (fill)
FILL(5)
 3:3 5:5
549:    O: O1126 (empty)
EMPTY(3)
 3:0 5:5
550:    O: O1129 (empty)
EMPTY(5)
 3:0 5:0
551:    O: O1131 (fill)
FILL(3)
 3:3 5:0
552:    O: O1133 (pour)
POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
553:    O: O1135 (empty)
EMPTY(5)
 3:0 5:0
554:    O: O1138 (fill)
FILL(3)
 3:3 5:0
555:    O: O1140 (pour)
POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
556:    O: O1143 (fill)
FILL(5)
 3:0 5:5
557:    O: O1145 (fill)
FILL(3)
 3:3 5:5
558:    O: O1148 (empt

764:    O: O1586 (fill)
FILL(3)
 3:3 5:5
765:    O: O1588 (empty)
EMPTY(5)
 3:3 5:0
766:    O: O1589 (empty)
EMPTY(3)
 3:0 5:0
767:    O: O1592 (fill)
FILL(3)
 3:3 5:0
768:    O: O1591 (fill)
FILL(5)
 3:3 5:5
769:    O: O1594 (empty)
EMPTY(3)
 3:0 5:5
770:    O: O1597 (fill)
FILL(3)
 3:3 5:5
771:    O: O1595 (empty)
EMPTY(5)
 3:3 5:0
772:    O: O1598 (empty)
EMPTY(3)
 3:0 5:0
773:    O: O1601 (fill)
FILL(3)
 3:3 5:0
774:    O: O1602 (pour)
POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
775:    O: O1605 (fill)
FILL(5)
 3:0 5:5
776:    O: O1607 (fill)
FILL(3)
 3:3 5:5
777:    O: O1609 (empty)
EMPTY(5)
 3:3 5:0
778:    O: O1610 (empty)
EMPTY(3)
 3:0 5:0
779:    O: O1612 (fill)
FILL(5)
 3:0 5:5
780:    O: O1613 (fill)
FILL(3)
 3:3 5:5
781:    O: O1615 (empty)
EMPTY(5)
 3:3 5:0
782:    O: O1617 (pour)
POUR(3:3,5:0) POUR(3:0,5:3)
 3:0 5:3
783:    O: O1619 (empty)
EMPTY(5)
 3:0 5:0
784:    O: O1623 (fill)
FILL(5)
 3:0 5:5
785:    O: O1624 (pour)
POUR(5:5,3:0) POUR(5:2,3:3)
 3:3 5:2
786:    O: O1628 (em

Run the above cell a few times. Sometimes it'll take only a few steps to reach the goal state, other times it'll take hundred. We need a way for the agent to search for the goal more intelligently.


## 4.11 Search Control

If you've spent time on search algorithms, you are probably familiar with closed lists. They're a way of tracking visited states to ensure that a search algorithm doesn't go to the same states over and over again. 

Soar has a mechanism to help avoid revisiting states. This is discussed later in the planning section.

For now, we will create a few simply heuristics to guide the search. 

Things that we don't want to happen.
* Filling a jug right after it has been emptied
* Pouring water from jugA to jugB then back to jugA

We can model this by preventing the agent from undoing the last applied operator.

We'll need new state to keep track of the last operator.
```
If the task is water-jug and the pour operator is selected, 
then create an augmentation of the state (last-operator) with the name of the operator 
    and a copy of the augmentations augmentation.
```

In [26]:
record_last_operator = """

sp {water-jug*apply*operator*record*last-operator*pour 
    (state <s> ^name water-jug
               ^operator <o>) 
    (<o> ^name      pour
         ^fill-jug  <fj>
         ^empty-jug <ej>) 
-->
    (<s> ^last-operator <last-op>) 
    (<last-op> ^name      pour
               ^fill-jug  <fj> 
               ^empty-jug <ej>)}

"""

TODO Finish